<div style='background-image: url("../share/Aerial_view_LLNL.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: center ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.8) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">Notebook 6:</div>
            <div style="font-size: x-large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Document Term Matrix and Textual Similarity</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Estimated Time: 45 minutes.</div>
        </div>
    </div>
</div>






# Textual Similarity

## Bag of Words (BoW) language model

Today we'll see our first, admittedly primitive, computational model of language called "Bag of Words". This model was very popular in early text analysis, and continues to be used today. In fact, the models that have replaced it are still very difficult to actually interpret, giving the BoW approach a slight advantage if we want to understand why the model makes certain decisions.

Getting into the model we'll have to revisit Term Frequency (think `Counter`). We'll then see the Document-Term Matrix (DTM), which we've discusssed briefly before. We'll have to normalize these counts if we want to compare. Then we'll look at the available Python libraries to streamline this process.

Once we have our BoW model we can analyze it in a high-dimensional vector space, which gives us more insights into the similarities and clustering of different texts.

Let's read in Augustine's *Confessions* text:

In [ ]:
with open('data/Augustine-Confessions.txt') as f:
    confessions = f.read()

print(confessions[:500])

There should be 13 books, which are fortunately separated by six line breaks:

In [ ]:
confessions_list = confessions.split('\n'*6)
len(confessions_list)

Let's peek at the first:

In [ ]:
print(confessions_list[0])

# Term Frequency Revisited

We'll remember from last week, that while `split` might be a quick way to get tokens, it's not the most accurate because it doesn't separate punctuation and contractions. We'll use `spacy` again to get tokens.

In [ ]:
import spacy

nlp = spacy.load('en', parser=False) 

In [ ]:
first_book = confessions_list[0]
parsed = nlp(first_book)
first_token_list = [token.text for token in parsed]
first_token_list[:500]

Now we can use `Counter` to get the term frequency:

In [ ]:
from collections import Counter
word_freq = Counter(first_token_list)
word_freq.most_common(20)

## Challenge

Write some code to get the 20 most common words of the second book. How similar are they to those of the first book?

# Document-Term Matrix

If we plan to compare word frequencies across texts, we could collate these `Counter` dictionaries for each book in `Confessions`. But we don't want to write all that code! There is an easy function that streamlines the process called `CountVectorizer`.

Let's look at the docstring:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
CountVectorizer?

Cool. So we'll create the `CountVectorizer` object, then transform it on our `list` of documents, here that would be the books in Augustine's `Confessions`.

In [ ]:
cv = CountVectorizer()
dtm = cv.fit_transform(confessions_list)
dtm

What's this? A sparse matrix just means that some cells in the table don't have value. Why? Because the vocabulary base is not the same for all the books! Let's try to demonstrate this.

In [ ]:
import pandas as pd

# de-sparsify
desparse = dtm.toarray()

# create labels for columns
word_list = cv.get_feature_names()

# create a new table
dtm_df = pd.DataFrame(columns=word_list, data=desparse)
dtm_df

Welcome to the ***Document Term Matrix***. This is a core concept in NLP and text analysis. It's not that complicated!

We have columns for each word *in the entire corpus*. Then each *row* is for each *document*. In our case, that's books in *Confessions*. The values are the word count for that word in the corresponding document. Note that there are many 0s, that word just doesn't show up in that document!

We can call up frequencies for a given word for each chapter easily, since they are the column names:

In [ ]:
dtm_df['read']

Looks to be about 13 counts, one for each book, let's double check!

In [ ]:
len(dtm_df['read'])

# Normalization

Let's take this another step further. In order to make apples-to-apples comparisons across Books, we can normalize our values by dividing each word count by the total number of words in its Book. To do that, we'll need to `sum` on `axis=1`, which means summing the row (number of words in that book), as opposed to summing the column.

Once we have the total number of words in that Book, we can get the percentage of words that one particular word accounts for, and we can do that for every word across the matrix!

In [ ]:
import numpy as np

row_sums = np.sum(desparse, axis=1)
normed = desparse/row_sums[:,None]
dtm_df = pd.DataFrame(columns=word_list, data=normed)
dtm_df

Reading the matrix above, we see that the word "abandoned" accounts for .0147% of words in Book 1, and .0278% of words in Book 2.

We can still grab out the normalized frequencies of the word 'read' for each book:

In [ ]:
dtm_df['abandoned']

For a variety of reasons we like to remove words like "the", "of", "and", etc.

In [ ]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

ENGLISH_STOP_WORDS

Since we are using an older translation of Augustine, we have to remove archaic forms of these stopwords as well.

In [ ]:
ye_olde_stop_words = ['thou','thy','thee', 'thine', 'ye', 'hath','hast', 'wilt','aught',\
                      'art', 'dost','doth', 'shall', 'shalt','tis','canst','thyself',\
                     'didst', 'yea', 'wert']

stop_words = list(ENGLISH_STOP_WORDS) + ye_olde_stop_words
stop_words

Let's re-run our code above, but feed in the `stop_words` to the `CountVectorizer`:

In [ ]:
cv = CountVectorizer(stop_words=stop_words)
dtm = cv.fit_transform(confessions_list)
desparse = dtm.toarray()
word_list = cv.get_feature_names()
dtm_df = pd.DataFrame(columns=word_list, data=desparse)
row_sums = np.sum(desparse, axis=1)
normed = desparse/row_sums[:,None]
dtm_df = pd.DataFrame(columns=word_list, data=normed)
dtm_df

# Streamlining

That was a lot of work, if this is such a common task hasn't someone streamlined this? In fact, we can simply instruct `CountVectorizer` not to include stopwords at all and another function, `TfidfTransformer`, normalizes easily.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

cv = CountVectorizer(stop_words=stop_words)
dtm = cv.fit_transform(confessions_list)
tt = TfidfTransformer(norm='l1',use_idf=False)
dtm_tf = tt.fit_transform(dtm)

# Vector Space Model of Language

Great, now we have a matrix with normalized frequencies of all the words ***in the entire corpus***. Right now our corpus is just all the books in Augustine's *Confessions*.

Let's move away from the table and just create a list of 13 vectors with only the normalized frequency values, one for each Book.

In [ ]:
dtm_array = dtm_tf.toarray()
dtm_array

Each vector has a number of coordinates equal to the number of unique words in the corpus. Let's just take Book 1:

In [ ]:
dtm_array[0]

One way to measure the similarity of texts, which Piper uses in his article, would be to measure the *Euclidean distance* between their coordinates in space. According to Wikipedia:

>The Euclidean distance or Euclidean metric is the "ordinary" straight-line distance between two points in Euclidean space

>$\mathrm{d}(\mathbf{b},\mathbf{a})=\sqrt{(a_1-b_1)^2 + (a_2-b_2)^2}$

Let's consider a simple 2 dimensional model. We have two point in space:

In [ ]:
a = (2,6)
b = (5,10)

euc_dist = np.sqrt( (a[0]-b[0])**2  +  (a[1]-b[1])**2 )
euc_dist

We can visualize this too:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter([a[0], b[0]], [a[1], b[1]])
plt.plot([a[0], b[0]], [a[1], b[1]])
plt.show()

We can think of this 2 dimensional distance between 2 points as looking at 2 different texts. In this *very* simple 2-d model though, we only have 2 words in the entire corpus! `(2,6)` and `(5,10)` would be the absolute counts for each text. Imagine:

```
Document 1:

the dog the dog dog dog dog dog

Document 2:

the dog the dog the dog the dog the dog dog dog dog dog dog

```

That would yield the comparison above. If we added a third point (document), we could see which 2 documents were closest to one another!

---

Ok, not too bad, but how do we do this with hundreds or thousands of dimensions (words) acorss hundreds or thousands of points (documents)? Well it actually scales the same way! Here it is for 3 dimensions:

$\mathrm{d}(\mathbf{b},\mathbf{a})=\sqrt{(a_1-b_1)^2 + (a_2-b_2)^2 + (a_3-b_3)^2}$

In [ ]:
a = (2,6,15)
b = (5,10,3)

euc_dist = np.sqrt( (a[0]-b[0])**2 +  (a[1]-b[1])**2 + (a[2]-b[2])**2 )
euc_dist

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter([a[0], b[0]], [a[1], b[1]], [a[2], b[2]])
ax.plot([a[0], b[0]], [a[1], b[1]], [a[2], b[2]])
plt.show()

We don't have to use our cool formula to calculate this, or to scale it up for *n* dimensions. That's what `scipy` is for:

In [ ]:
from scipy.spatial import distance
distance.euclidean(a,b)

---

Another measure of two vectors, more common for text analysis, is called *cosine similarity*. According to Wikipedia:

>Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. The cosine of 0° is 1, and it is less than 1 for any other angle. It is thus a judgment of orientation and not magnitude: two vectors with the same orientation have a cosine similarity of 1, two vectors at 90° have a similarity of 0, and two vectors diametrically opposed have a similarity of -1, independent of their magnitude.

>$\text{similarity} = \cos(\theta) = {\mathbf{A} \cdot \mathbf{B} \over \|\mathbf{A}\|_2 \|\mathbf{B}\|_2} = \frac{ \sum\limits_{i=1}^{n}{A_i  B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{A_i^2}}  \sqrt{\sum\limits_{i=1}^{n}{B_i^2}} }$

Essentially we want to take the cosine of the angle formed between two vectors (documents). We start the vector at the origin and measure the angle between the two vectors we're interested in.

In [ ]:
mpl.rcParams['legend.fontsize'] = 10

origin = (0,0,0)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter([a[0], b[0], origin[0]], [a[1], b[1], origin[1]], [a[2], b[2], origin[2]])
ax.plot([origin[0], a[0]], [origin[1], a[1]], [origin[2], a[2]])
ax.plot([origin[0], b[0]], [origin[1], b[1]], [origin[2], b[2]])
plt.show()

Let's go back to two dimensions for the vanilla `numpy` calculation:

In [ ]:
a = (2,6)
b = (5,10)

# don't worry about the formula so much as the intuition behind it: angle between vectors
cos_dist = 1 - (a[0]*b[0] + a[1]*b[1]) / ( np.sqrt(a[0]**2 + a[1]**2 ) * np.sqrt(b[0]**2 + b[1]**2 ) )
cos_dist

Of course, `scipy` has taken care of this for us too:

In [ ]:
distance.cosine(a,b)

For the 3-d model:

In [ ]:
a = (2,6,15)
b = (5,10,3)
distance.cosine(a,b)

## Challenge

Try passing different values into both the euclidean and cosine distance functions. What is your intuition about these different measurements? Remember that all values in the Term-Frequency Matrix are positive, between [0,1], and that most are very small.

# Visualizing Texts in Vector Space

Let's walk through this now. Say we have 3 texts, `a`, `b`, and `c`. The whole corpus, again, only has 2 words (dimensions)!

In [ ]:
a = (2,6)
b = (5,10)
c = (14,11)

print(distance.euclidean(a,b))
print(distance.euclidean(a,c))
print(distance.euclidean(b,c))

We'll make a matrix for the points:

In [ ]:
point_matrix = np.array([a,b,c])
point_matrix

Now we can use `sklearn`'s `pairwise_distances` method to compare each book to each book:

In [ ]:
from sklearn.metrics import pairwise
pairwise.pairwise_distances(point_matrix, metric='euclidean')

Cool! We got what we calculated. Note: the results are mirrored because the columns and rows are both the same texts.

We can do the same thing on Augustine's *Confessions*, remember the rows are for each Book too!:

In [ ]:
dist_matrix = pairwise.pairwise_distances(dtm_tf, metric='euclidean')

title_list = ['Book '+str(i+1) for i in range(len(confessions_list))]
pd.DataFrame(columns=title_list, data=dist_matrix)

Visualizing hundreds of dimensions is difficult for us. So we can use multi-dimensional scaling (MDS) to put this into a 2-d graph for us:

In [ ]:
from sklearn.manifold import MDS

mds = MDS(n_components = 2, dissimilarity="precomputed")
embeddings = mds.fit_transform(dist_matrix)

_, ax = plt.subplots(figsize=(10,10))
ax.scatter(embeddings[:,0], embeddings[:,1], alpha=0)
for i in range(13):
    ax.annotate(i+1, ((embeddings[i,0], embeddings[i,1])))

# Brief Aside: K-Means Clustering

Tries to find natural groupings among points, once we tell it how many groups to look for.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2)
kmeans.fit_predict(dist_matrix)